#### Get composite data from WRDS
Save as "sp500_cons.csv"

In [1]:
import pandas as pd
import wrds

conn = wrds.Connection(wrds_username='xiaolou')
sp500 = conn.raw_sql("""select a.*, b.date, b.ret
                        from crsp.msp500list as a, crsp.msf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/01/1926'
                        order by date;
                        """, date_cols=['start', 'ending', 'date'])
sp500.to_csv("sp500_cons.csv", index=False)
conn.close()

ModuleNotFoundError: No module named 'wrds'

#### Generate New Panel for Constituents with 2021 Data
Read in sp500_constituent_monthly_total_rtns_2021.xlsx, permno_aft_revise.csv. Output panel with 2021 to panel_cons.csv.

In [ ]:
data = pd.read_excel("./sp500_constituent_monthly_total_rtns_2021.xlsx")
permno_list = pd.read_csv('./permno_aft_revise.csv')[['ticker', 'permno']].drop_duplicates(subset=['ticker'])
permno_list.permno = permno_list['permno'].astype(int).astype(str)
data['comp'] = data['comp'].apply(lambda x: x.split(' ')[0])
data = pd.merge(data, permno_list, how='left', left_on='comp', right_on='ticker')
data = data.dropna(subset=['permno'])
data.index = data['permno']
data = data.iloc[:, 1:-2]
new = None
n = data.shape[0]
for i, dt in enumerate(data.columns):
    if new is None:
        new = pd.DataFrame({'permno': data.index, 'date': [dt]*n, 'ret': data[dt]})
    else:
        temp = pd.DataFrame({'permno': data.index, 'date': [dt]*n, 'ret': data[dt]})
        new = new.append(temp)

dta = pd.read_csv('./sp500_cons.csv')
# treatment of the data
## turn permno into str
dta['permno'] = dta['permno'].astype(int).astype(str)
## change date into pandas date format
dta['date'] = pd.to_datetime(dta['date'], format='%Y-%m-%d')
## keep permno, date, ret
dta = dta[['date', 'permno', 'ret']]
dta = dta.append(new)
## drop duplicates
dta = dta.drop_duplicates(subset=['date', 'permno'])
## missing return labelled as -66, -77, -88, -99
dta = dta[dta['ret'] != -66]
dta = dta[dta['ret'] != -77]
dta = dta[dta['ret'] != -88]
dta = dta[dta['ret'] != -99]
## drop nan
dta = dta.dropna()
print(dta.shape)
# generate permno list
df = None
permno = dta['permno'].unique()
for pmo in permno:
    if df is None:
        df = dta[dta['permno'] == pmo]
        df[pmo] = df['ret']
        df = df[['date', pmo]]
    else:
        new = dta[dta['permno'] == pmo]
        new[pmo] = new['ret']
        new = new[['date', pmo]]
        df = pd.merge(df, new, how='outer', on='date')
df.to_csv('panel_cons.csv', index=False)

#### Turn Composite Data Into Panel Form
Save as "panel_cons.csv"

In [ ]:
dta = pd.read_csv('./sp500_cons.csv')
# treatment of the data
## turn permno into str
dta['permno'] = dta['permno'].astype(int).astype(str)
## change date into pandas date format
dta['date'] = pd.to_datetime(dta['date'], format='%Y-%m-%d')
## keep permno, date, ret
dta = dta[['date', 'permno', 'ret']]
## drop duplicates
dta = dta.drop_duplicates(subset=['date', 'permno'])
## missing return labelled as -66, -77, -88, -99
dta = dta[dta['ret'] != -66]
dta = dta[dta['ret'] != -77]
dta = dta[dta['ret'] != -88]
dta = dta[dta['ret'] != -99]
## drop nan
dta = dta.dropna()
print(dta.shape)

# generate permno list
df = None
permno = dta['permno'].unique()
for pmo in permno:
    if df is None:
        df = dta[dta['permno'] == pmo]
        df[pmo] = df['ret']
        df = df[['date', pmo]]
    else:
        new = dta[dta['permno'] == pmo]
        new[pmo] = new['ret']
        new = new[['date', pmo]]
        df = pd.merge(df, new, how='outer', on='date')
df.to_csv('panel_cons.csv', index=False)

#### Calculate Annual Return For Composites
Save as "./ann_panel.csv"

In [ ]:
df = pd.read_csv('./panel_cons.csv')

df.index = pd.to_datetime(df['date'], format='%Y-%m-%d')
df = df.iloc[:, 1:]
df = df + 1
df = df.cumprod()
df['year'] = df.index.year
annual = pd.DataFrame(columns=df.columns)
ann_bef = None
start_year = min(df['year'])
end_year = max(df['year'])
for i in range(start_year, end_year + 1):
    ann = df.loc[df.year == i]
    if i == start_year:
        ann = ann.iloc[-1, :] - 1
    else:
        ann = ann.iloc[-1, :] / ann_bef - 1
    ann_bef = df.loc[df.year == i].iloc[-1, :]
    ann['year'] = i
    annual = annual.append(ann, ignore_index=True)

annual['year'] = annual['year'].astype(int)
annual.to_csv("./ann_panel.csv", index=False)

#### Count Up and Down Month for Composites
Save as "summary_for_cons.csv"

In [ ]:
"""CALCULATE UP AND DOWN MONTH"""
df = pd.read_csv('./panel_cons.csv')
df.index = pd.to_datetime(df['date'], format='%Y-%m-%d')
df = df.iloc[:, 1:]     # get rid of date as a column

# select negative return month
nega_df = df.fillna(value=100)
nega_df = (nega_df < 0)
nega_df = pd.DataFrame(nega_df.sum(axis=0), columns=['downMonth'])
# select positive return month
pos_df = df.fillna(value=-100)
pos_df = (pos_df >= 0)
pos_df = pd.DataFrame(pos_df.sum(axis=0), columns=['upMonth'])

"""CALCULATE UP AND DOWN YEAR"""
df = pd.read_csv('./ann_panel.csv')
df.index = df.year
df = df.iloc[:, :-1]

# select negative return month
nega_df_y = df.fillna(value=100)
nega_df_y = (nega_df_y < 0)
nega_df_y = pd.DataFrame(nega_df_y.sum(axis=0), columns=['downYear'])
# select positive return month
pos_df_y = df.fillna(value=-100)
pos_df_y = (pos_df_y >= 0)
pos_df_y = pd.DataFrame(pos_df_y.sum(axis=0), columns=['upYear'])

"""MERGE THE SERIES"""
result = pd.merge(nega_df_y, pos_df_y, how='outer', left_index=True, right_index=True)
result = pd.merge(result, nega_df, how='outer', left_index=True, right_index=True)
result = pd.merge(result, pos_df, how='outer', left_index=True, right_index=True)
result.to_csv("./summary_for_cons.csv", index=True)

#### Calculate Annual Return and Summary Data for Index
Use "index_sp500.csv", annual return save as "index_ann.csv", summary statistics save as "summary_for_ix.csv"

In [ ]:
df = pd.read_csv('./index_sp500.csv')
df = df.dropna()
df['date'] = pd.to_datetime(df['caldt'], format='%Y%m%d')
df['ret'] = df['vwretd']
df = df[['date', 'ret']]

"""CALCULATE ANNUAL RETURN"""
dta = df.copy(deep=True)
dta['ret'] = dta['ret'] + 1
dta['ret'] = dta['ret'].cumprod()
dta['ret_lag'] = dta['ret'].shift(12)
dta['ann_ret'] = dta['ret'] / dta['ret_lag'] - 1
dta.iloc[11, 3] = dta.iloc[11, 1] - 1
dta['month'] = dta.date.apply(lambda x: x.month)
dta['year'] = dta.date.apply(lambda x: x.year)

dta = dta[dta['month'] == 12]
dta = dta[['year', 'ann_ret']]
dta.to_csv("./index_ann.csv", index=False)

"""CALCULATE MONTH"""
result = dict()
result['downMonth'] = (df['ret'] < 0).sum()
result['upMonth'] = (df['ret'] >= 0).sum()
result['downYear'] = (dta['ann_ret'] < 0).sum()
result['upYear'] = (dta['ann_ret'] >= 0).sum()

consecUp, consecDown = 0, 0
currUp, currDown = 0, 0
for i in range(df.shape[0]):
    curr = df.iloc[i, 1]
    if curr is None:
        continue
    else:
        if curr >= 0:
            currUp += 1
            consecUp = max(consecUp, currUp)
            currDown = 0
        else:
            currUp = 0
            currDown += 1
            consecDown = max(consecDown, currDown)
result['consecUp'] = consecUp
result['consecDown'] = consecDown
pGivep, pGiven, nGivep, nGiven = 0, 0, 0, 0
for i in range(1, df.shape[0]):
    now = df.iloc[i, 1]
    prev = df.iloc[i - 1, 1]
    if now is None or prev is None:
        continue
    if now >= 0 and prev >= 0:
        pGivep += 1
    elif now >= 0 and prev < 0:
        pGiven += 1
    elif now < 0 and prev < 0:
        nGiven += 1
    elif now < 0 and prev >= 0:
        nGivep += 1
result['PosAftPos'] = pGivep
result['PosAftNeg'] = pGiven
result['NegAftPos'] = nGivep
result['NegAftNeg'] = nGiven
pd.DataFrame(result, index=[0]).to_csv('summary_for_ix.csv', index=False)

#### Calculate up and down company number for each year and each month
summary statistics save as "summary_by_year.csv" and "summary_by_month.csv"

In [ ]:
"""CALCULATE UP AND DOWN MONTH"""
df = pd.read_csv('./panel_cons.csv')
df.index = pd.to_datetime(df['date'], format='%Y-%m-%d')
df = df.iloc[:, 1:]     # get rid of date as a column
# select negative return month
nega_df = df.fillna(value=100)
nega_df = (nega_df < 0)
nega_df = pd.DataFrame(nega_df.sum(axis=1), columns=['downComp'])
# select positive return month
pos_df = df.fillna(value=-100)
pos_df = (pos_df >= 0)
pos_df = pd.DataFrame(pos_df.sum(axis=1), columns=['upComp'])

result_m = pd.merge(nega_df, pos_df, how='outer', left_index=True, right_index=True)
result_m.to_csv("./summary_by_month.csv")

"""CALCULATE UP AND DOWN YEAR"""
df = pd.read_csv('./ann_panel.csv')
df.index = df.year
df = df.iloc[:, :-1]
# select negative return year
nega_df_y = df.fillna(value=100)
nega_df_y = (nega_df_y < 0)
nega_df_y = pd.DataFrame(nega_df_y.sum(axis=1), columns=['downComp'])
# select positive return year
pos_df_y = df.fillna(value=-100)
pos_df_y = (pos_df_y >= 0)
pos_df_y = pd.DataFrame(pos_df_y.sum(axis=1), columns=['upComp'])

result_y = pd.merge(nega_df_y, pos_df_y, how='outer', left_index=True, right_index=True)
result_y.to_csv("./summary_by_year.csv")
print(nega_df, pos_df, nega_df_y, pos_df_y)

#### Calculate Consecutive Month Number for Up and Down Periods
Save as "summary_for_cons2.csv"

In [ ]:
def longestUpMonth(series):
    monthMax = 0
    curr = 0
    for i in series:
        if i is None:
            curr = 0
        elif i >= 0:
            curr += 1
            monthMax = max(monthMax, curr)
        else:
            curr = 0
    return monthMax


def longestDownMonth(series):
    monthMax = 0
    curr = 0
    for i in series:
        if i is None:
            curr = 0
        elif i < 0:
            curr += 1
            monthMax = max(monthMax, curr)
        else:
            curr = 0
    return monthMax

df = pd.read_csv('./panel_cons.csv')
df.index = pd.to_datetime(df['date'], format='%Y-%m-%d')
df = df.iloc[:, 1:]     # get rid of date as a column
result = []
for i, comp in enumerate(df.columns):
    numUp = longestUpMonth(df[comp])
    numDown = longestDownMonth(df[comp])
    result.append([comp, numUp, numDown])
result = pd.DataFrame(result, columns=['permno', 'consecUp', 'consecDown'])
result.index = result.permno
result = result[['consecUp', 'consecDown']]
dta1 = pd.read_csv('summary_for_cons.csv', index_col=0)
dta1.index = dta1.index.astype(str)
dta1 = pd.merge(dta1, result, how='outer', right_index=True, left_index=True)
dta1.to_csv('summary_for_cons2.csv')

#### Calculate Conditional Up and Down Time
Save as 'summary_for_cons3.csv'

In [ ]:
df = pd.read_csv('./panel_cons.csv')
df.index = pd.to_datetime(df['date'], format='%Y-%m-%d')
df = df.iloc[:, 1:]

result = []
for j, comp in enumerate(df.columns):
    pGivep, pGiven, nGivep, nGiven = 0, 0, 0, 0
    for i in range(1, df.shape[0]):
        now = df.iloc[i, j]
        prev = df.iloc[i-1, j]
        if now is None or prev is None:
            continue
        if now >= 0 and prev >= 0:
            pGivep += 1
        elif now >= 0 and prev < 0:
            pGiven += 1
        elif now < 0 and prev < 0:
            nGiven += 1
        elif now < 0 and prev >= 0:
            nGivep += 1
    result.append([comp, pGivep, pGiven, nGivep, nGiven])
result = pd.DataFrame(result, columns=['permno', 'PosAftPos', 'PosAftNeg', 'NegAftPos', 'NegAftNeg'])
result.index = result.permno
result = result[['PosAftPos', 'PosAftNeg', 'NegAftPos', 'NegAftNeg']]

dta2 = pd.read_csv('summary_for_cons2.csv', index_col=0)
dta2.index = dta2.index.astype(str)
dta2 = pd.merge(dta2, result, how='outer', right_index=True, left_index=True)
dta2.to_csv('summary_for_cons3.csv')